In [21]:
import pandas as pd
from tqdm import tqdm
import os

# CONNECT to REDSHIFT

In [22]:
from sqlalchemy import create_engine
import psycopg2
import dotenv

from os.path import expanduser
home = expanduser("~")

dotenv.load_dotenv(fr"{home}\creds\local_postgres.txt")

connection_params = dict(
        host=os.getenv('host')
        , dbname=os.getenv('dbname')
        , user=os.getenv('user')
        , password=os.getenv('password')
        , port=os.getenv('port')
)

week_schema = 'week7'

engine = create_engine(f'postgresql+psycopg2://{os.getenv("user")}:{os.getenv("password")}@{os.getenv("host")}:{os.getenv("port")}/{os.getenv("dbname")}')
engine.connect()

OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (::1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?

(Background on this error at: https://sqlalche.me/e/14/e3q8)

# Read CSVs into dataframes and upload

In [19]:
# walk through Tablas subdirectory and read all csv files into list of dataframes
df_dict = {}
for filename in tqdm(os.listdir('Tablas')):
    if filename.endswith('.csv'):
        pretty_filename = filename[:-4]
        df_dict[pretty_filename] = pd.read_csv('./Tablas/' + filename)
        df_dict[pretty_filename].to_sql(pretty_filename, engine, index=False, if_exists='replace', schema=week_schema)

  0%|          | 0/12 [00:00<?, ?it/s]


PendingRollbackError: Can't reconnect until invalid transaction is rolled back. (Background on this error at: https://sqlalche.me/e/14/8s2b)

In [42]:
df_dict.keys()

dict_keys(['countryregioncurrency', 'currencyrate', 'product', 'productcategory', 'productdescription', 'productmodelproductdescriptionculture', 'productreview', 'productsubcategory', 'salesorderdetail', 'salesorderheader', 'salesperson', 'salesterritory'])

In [33]:

stmt = f"""
SELECT * 
FROM week7.product p 
INNER JOIN week7.productreview pr
ON p.productid = pr.productid

"""

reviews = pd.read_sql_query(stmt, engine)


In [60]:
# which are the top salesmen
stmt = f"""
SELECT soh.salespersonid, SUM(soh.subtotal) AS total_sales
FROM week7.salesorderheader soh
INNER JOIN week7.salesperson sp
ON soh.salespersonid = sp.businessentityid

GROUP BY soh.salespersonid
ORDER BY total_sales DESC
"""


salesmen_sales = pd.read_sql_query(stmt, engine)
salesmen_sales.head(10)


,salespersonid,total_sales
0,276.0,1.036701e+07
1,277.0,1.006580e+07
2,275.0,9.293903e+06
3,289.0,8.503339e+06
4,279.0,7.171013e+06
5,281.0,6.427006e+06
6,282.0,5.926418e+06
7,290.0,4.509889e+06
8,283.0,3.729945e+06
9,278.0,3.609447e+06


In [63]:
df_dict['product'].head(2)

,productid,name,productnumber,makeflag,finishedgoodsflag,color,safetystocklevel,reorderpoint,standardcost,listprice,size,sizeunitmeasurecode,weightunitmeasurecode,weight,daystomanufacture,productline,class,style,productsubcategoryid,productmodelid,sellstartdate,sellenddate,discontinueddate,rowguid,modifieddate
0,1,Adjustable Race,AR-5381,f,f,NaN,1000,750,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,694215b7-08f7-4c0d-acb1-d734ba44c0c8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,f,f,NaN,1000,750,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,58ae3c20-4f3a-4749-a7d4-d568806cc537,2014-02-08 10:01:36.827


In [55]:
# which are the top selling products?

stmt = f"""

SELECT p.productid, p.name, SUM(sod.orderqty)* AS total_qty
"""

,salesorderid,revisionnumber,orderdate,duedate,shipdate,status,onlineorderflag,purchaseordernumber,accountnumber,customerid,salespersonid,territoryid,billtoaddressid,shiptoaddressid,shipmethodid,creditcardid,creditcardapprovalcode,currencyrateid,subtotal,taxamt,freight,totaldue,comment,rowguid,modifieddate
0,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,f,PO522145787,10-4020-000676,29825,279.0,5,985,985,5,16281.0,105041Vi84182,NaN,20565.6206,1971.5149,616.0984,23153.2339,None,79b65321-39ca-4115-9cba-8fe0903e12e6,2011-06-07 00:00:00
1,43660,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,f,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738dc42d-d03b-48a1-9822-f95a67ea7389,2011-06-07 00:00:00
2,43661,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,f,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,d91b9131-18a4-4a11-bc3a-90b6f53e9d74,2011-06-07 00:00:00
3,43662,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,f,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4a1ecfc0-cc3a-4740-b028-1c50bb48711c,2011-06-07 00:00:00
4,43663,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,f,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9b1e7a40-6ae0-4ad3-811c-a64951857c4b,2011-06-07 00:00:00
